In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2024-10-27 14:55:02,164 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType
from pyspark.sql import functions as F
import re

In [3]:
spark = SparkSession.builder.appName("Clickstream").getOrCreate()

In [4]:
clickstream_df = spark.read.csv("hdfs:/data/clickstream.csv", header=True, inferSchema=True, sep='\t')
clickstream_df.createOrReplaceTempView("clickstream")

In [5]:
routes = spark.sql("""
with error_flag as
    (SELECT *, 
           CASE WHEN event_type LIKE '%error%' THEN 1 ELSE 0 END AS is_error 
    FROM clickstream),
    cleaned_events as
    (SELECT *
    FROM (
        SELECT *, 
               SUM(is_error) OVER(PARTITION BY user_id, session_id ORDER BY timestamp) AS error_flag
        FROM error_flag
    ) WHERE error_flag = 0 AND event_type = 'page'),
    events as
    (SELECT user_id, session_id, event_page, timestamp
    FROM (
        SELECT *,
               LAG(event_page) OVER (PARTITION BY user_id, session_id ORDER BY timestamp) AS prev_page
        FROM cleaned_events
    ) 
    WHERE event_page != prev_page OR prev_page IS NULL)
    SELECT user_id, session_id, 
           CONCAT_WS('-', COLLECT_LIST(event_page)) AS route
    FROM events
    GROUP BY user_id, session_id
    
    
""")

routes.createOrReplaceTempView("routes")

top_routes_df = spark.sql("""
    SELECT route, COUNT(*) AS count
    FROM routes
    GROUP BY route
    ORDER BY count DESC
    LIMIT 30
""")

In [12]:
with open("top30.txt", 'w') as file:
    for row in top_routes_df:
        file.write(f"{row.route}\t{row.count}\n")